# MI test

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('/Users/paolo/Documents/Droughts/scripts')


In [2]:
from mixedRVMI import MIEstimate

In [3]:
# importo pickle con il dataframe
# riduco a 100.000 dati per non essere troppo pesante 
# aggiungo una colonna non linearmente correlata e una indipendente
current_df = pd.read_pickle("/Users/paolo/Documents/Droughts/data/NDVI_2020052.pkl")
current_df = current_df.dropna(subset = ["SMN"])
current_df = current_df.iloc[0:100000]
current_df['SMN_2'] = current_df['SMN'].apply(lambda row: row*row)
current_df['SMN_ind'] = np.linspace(0,100,num=len(current_df['SMN']))
current_df.head()

,SMN,latitude,longitude,SMN_2,SMN_ind
97,0.003,71.514,-179.982,0.000009,0.000
98,0.001,71.478,-179.982,0.000001,0.001
99,0.001,71.442,-179.982,0.000001,0.002
100,0.012,71.406,-179.982,0.000144,0.003
101,0.012,71.370,-179.982,0.000144,0.004


In [14]:
print("MI(SMN,SMN^2) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df['SMN_2'].to_numpy(),estimate='digamma',k=3)))
print("MI(SMN,independent feature) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df['SMN_ind'].to_numpy(),estimate='digamma',k=3)))
print("MI(SMN,both) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df.loc[:,['SMN_2','SMN_ind']].to_numpy(),estimate='digamma',k=3)))

MI(SMN,SMN^2) = 4.946787949670897
MI(SMN,independent feature) = 0.9782831328718034
MI(SMN,both) = 1.1940409535125753
